# Poisson 1D

This notebook solves the linear problem:

$$ -u(x)'' = f(x) \text{ in } \Omega = [-1,1]$$ 

with

$$ f(x) = \frac{\pi^2}{4} \cos{\left(\frac{\pi x}{2}\right)} $$

and either homogeneous Dirichlet boundary conditions or mixed Dirichlet and Neumann using Hermite elements. The purpose is to find what the issues are while setting Dirichlet BCs and fix them. See [Manually rebuilding the mass matrix](#Manually-rebuilding-the-mass-matrix).

In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
import FIAT
%matplotlib inline
import nbimporter
from interpolation import make_constant

# Exact solution with homogeneous Dirichlet BCs:
u_exact = lambda x: np.cos(np.pi*x/2)

In [ ]:
def poisson1d(V:FunctionSpace, use_neumann:bool=False):
    """ Solves -u'' = f on [-1,1] with f=... """
    deg = V.ufl_element().degree()
    
    LEFT = V.mesh().coordinates().min()
    RIGHT = V.mesh().coordinates().max()

    u0 = make_constant(0., V) # Work around lack of evaluate_dofs() for Hermite

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4", degree=deg)

    a = u.dx(0)*v.dx(0)*dx
    L = f*v*dx 
    
    if use_neumann: # Note that this changes the exact solution!
        class Neumann(SubDomain):
            def inside(self, x, on_boundary):
                return on_boundary and near(x[0], RIGHT)
        NEUMANN_ID = 1
        exterior = FacetFunction('uint', V.mesh())
        neu_bdry = Neumann()
        neu_bdry.mark(exterior, NEUMANN_ID)
        bc = DirichletBC(V, u0, lambda x: near(x[0], LEFT))
        L = L - v*ds(subdomain_data=exterior, subdomain_id=NEUMANN_ID)
    else:
        bc = DirichletBC(V, u0, lambda x: near(x[0], LEFT) or near(x[0], RIGHT))

    u = Function(V)
    solve(a == L, u, bc)
    return u

In [ ]:
mesh = IntervalMesh(20, -1, 1)

V = FunctionSpace(mesh, "Lagrange", 3)
u_lag2 = poisson1d(V)

In [ ]:
xx = np.linspace(-1, 1, 100)
pl.plot(xx, [u_lag2(x) for x in xx], label='$u_{l}$')
#pl.plot(xx, [u_exact(x) for x in xx], label='$u_{ex}$')
pl.title("Solution with CG2 elements")
_ = pl.legend()

# Using Hermite elements

Naively applying Dirichlet boundary conditions fails because first derivatives are set to 0 when `DirichletBC` clears all the rows of dofs of nodes at the boundary, since this includes Hermite dofs associated to partial derivatives.

## A note about the solutions

Note that we always plot the solution by evaluating it at the mesh vertices. If instead we choose to evaluate it at arbitrary points in the domain, the solutions in Hermite spaces show a wiggly behaviour. Is this expected or an error? 

In [ ]:
V = FunctionSpace(mesh, "Hermite", 3)
u = poisson1d(V)

xx = V.mesh().coordinates() #np.linspace(-1, 1, 100)
pl.figure(figsize=(8,8))
pl.plot(xx, [u(x) for x in xx], label='$u_h$')
pl.plot(xx, [u_lag2(x) for x in xx], label='$u_{l}$')
_ = pl.legend()

## Manually rebuilding the stiffness matrix

The solution is to manually restore the relevant rows of the assembled matrix to their values before `DirichletBC.apply()` emptied them. The code has been factored out into [boundary.ipynb](boundary.ipynb).

Analogously, the way one applies Neumann boundary conditions has to be adapted in e.g. 4th order problems like the beam equation: in that case Neumann bcs are essential and are included in the function space by setting the relevant dofs to the desired values. See [Euler-Bernoulli.ipynb](Euler-Bernoulli.ipynb).

In [ ]:
import nbimporter
from boundary import *
from utils import fnor
from interpolation import interpolate_hermite, make_constant

def poisson1d_hermite(V, use_neumann:bool=False):
    """
        V: FunctionSpace.
    """
    assert V.ufl_element().family() == 'Hermite', 'Please pay attention.'

    _left = V.mesh().coordinates().min()
    _right = V.mesh().coordinates().max()

    def left(x):
        return near(x[0], _left)
    def right(x):
        return near(x[0], _right)

    u0 = make_constant(0, V)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("pi*pi*cos(pi*x[0]/2)/4",
                   degree=V.ufl_element().degree())

    a = u.dx(0)*v.dx(0)*dx
    F = f*v*dx 

    if use_neumann:
        dir_bc = DirichletBC(V, u0, left)
        class Right(SubDomain):
            def inside(self, x, on_boundary):
                return on_boundary and near(x[0], _right)

        exterior = FacetFunction('uint', V.mesh(), value=0)
        neu_bdry = Right()
        neu_bdry.mark(exterior, 1)
        F = F - 1.0*v*ds(subdomain_data=exterior, subdomain_id=1)
    else:
        dir_bc = DirichletBC(V, u0, fnor(left, right))

    A = assemble(a)
    b = assemble(F)
    apply_dirichlet_hermite(A, b, dir_bc)    
    u = Function(V)
    U = u.vector()
    solve(A, U, b)

    return u

In [ ]:
mesh = IntervalMesh(10, -1, 1)
V = FunctionSpace(mesh, 'Hermite', 3)
u_her = poisson1d_hermite(V)

In [ ]:
xx = V.mesh().coordinates() # np.linspace(-1,1,100)
pl.figure(figsize=(12,6))
pl.plot(xx, [u_her(x) for x in xx], '-', label='$u_h$')
pl.plot(xx, [u_lag2(x) for x in xx], '-.', label='$u_{lag2}$')#, alpha=0.7)
_ = pl.legend()